Predict if a mail is a spam or not.

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than 5 and using character n-grams from n=2 to n=5.

Use this document-term matrix and the following additional features:
- the length of document (number of characters)
- number of digits per document
- number of non-word characters (anything other than a letter, digit or underscore.)
- fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also find the 10 smallest and 10 largest feature from the model and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients: ['length_of_doc', 'digit_count', 'non_word_char_count']

This function should return a tuple (AUC score as a float, smallest coefs list, largest coefs list).

In [39]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

spam_data = pd.read_csv('spam.csv')
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [40]:
# Creating Train and test sets
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [41]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [42]:
def predictor():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    
    # transform texts into a document-term matrix
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # get the len of each X_train as a feature
    X_train_len = [len(x) for x in X_train]
    X_test_len = [len(x) for x in X_test]
    
    # get the number of digits for of each document
    X_train_n_digit = [len(re.findall('\d', x)) for x in X_train]
    X_test_n_digit = [len(re.findall('\d', x)) for x in X_test]
    
    # get the number of non word chars for of each document
    X_train_n_nwords = [len(re.findall('\W', x)) for x in X_train]
    X_test_n_nword = [len(re.findall('\W', x)) for x in X_test]

    # Enrich the train and test data with the new feature
    X_train_enriched = add_feature(X_train_vectorized, X_train_len)
    X_train_enriched = add_feature(X_train_enriched, X_train_n_digit)
    X_train_enriched = add_feature(X_train_enriched, X_train_n_nwords)

    X_test_enriched = add_feature(X_test_vectorized, X_test_len)
    X_test_enriched = add_feature(X_test_enriched, X_test_n_digit)
    X_test_enriched = add_feature(X_test_enriched, X_test_n_nword)
    
    # Train a Logistic model
    model = LogisticRegression(max_iter=100)
    model.fit(X_train_enriched, y_train)

    # Predict test documents
    predictions = model.predict(X_test_enriched)
    
    feature_names = vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count']
    s1 = pd.Series(model.coef_[0], index=feature_names)
    
    return roc_auc_score(y_test, predictions), s1.nsmallest(10).index.tolist(), s1.nlargest(10).index.tolist()

In [45]:
result = predictor()
print("AUC score: {}\nTop ten smallest feature: {}\nTop ten largest feature: {}".format(result[0], result[1], result[2]))

AUC score: 0.9759031798040846
Top ten smallest feature: [' i', 'ca', '..', '. ', ' m', 'pe', ' go', 'if', 'us', ' h']
Top ten largest feature: ['digit_count', 'xt', 'co', ' r', 'ne', 'ia', ' ba', ' x', 'ian ', 'ba']
